### Querying a created index

In [432]:
import json

# build dicts for product names and embs
product_names = {}
product_embs = {}
product_text = {}
with open("questions.json") as f:
    for l in f.readlines():
        p = json.loads(l)
        id = p["id"]
        product_names[id] = p["id"]
        product_text[id] = p['splitted_texts_chunks']
        product_embs[id] = p["embedding"]

In [433]:
# get the embedding for ID 6523 "cloudveil women's excursion short"
# you can also try with other IDs such as 12711, 18090, 19536 and 11863
query_emb = product_embs["0"]

In [435]:
# run query
response = my_index_endpoint.find_neighbors(
    deployed_index_id=DEPLOYED_INDEX_ID, queries=[query_emb], num_neighbors=3
)

# show the results
for idx, neighbor in enumerate(response[0]):
    print(f"{neighbor.distance:.2f} {product_names[neighbor.id]} {product_text[neighbor.id]}")

1.00 0 IN WHICH
0.56 18 Digitized by
Google

0.56 1 We Are Introduced to
CHAPTER I
Winnie-the-Pooh and Some
Bees, and the Stories Begin
HERE is Edward B


### Run Query
Finally it's ready to use Vector Search. In the following code, it creates an embedding for a test question, and find similar question with the Vector Search.

In [166]:


test_embeddings = get_embeddings_wrapper(["How to read JSON with Python?"])
# Test query
response = my_index_endpoint.find_neighbors(
    deployed_index_id=DEPLOYED_INDEX_ID,
    queries=test_embeddings,
    num_neighbors=20,
)

# show the result
import numpy as np

for idx, neighbor in enumerate(response[0]):
    id = np.int64(neighbor.id)
    similar = df.query("id == @id", engine="python")
    print(f"{neighbor.distance:.4f} {similar.title.values[0]}")


100%|██████████| 1/1 [00:01<00:00,  1.10s/it]


TypeError: Value must be iterable

### Get an existing Index
To get an index object that already exists, replace the following [your-index-id] with the index ID and run the cell. You can check the ID on the Vector Search Console > INDEXES tab.


In [167]:
# my_index_id = "vs-quickstart-index-endpoint-02051523"  # @param {type:"string"}
# my_index = aiplatform.MatchingEngineIndex(my_index_id)
del(my_index)
my_index = aiplatform.MatchingEngineIndex(
    index_name='projects/255766800726/locations/us-central1/indexes/5517934288365944832'
)

my_index_endpoint_id = "projects/255766800726/locations/us-central1/indexEndpoints/3345510418113101824"

# my_index_endpoint_id = "[your-index-endpoint-id]"  # @param {type:"string"}
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(my_index_endpoint_id)


#### Querying the earlier created index

In [225]:
# this is embedding vector (should be created by calling the embeddings models)

text = "This is a test document."

test_embeddings = embeddings.embed_query(text)
print("preview embeddings",test_embeddings[0:2])

preview embeddings [-0.009106644429266453, 0.008858383633196354]


In [223]:
# this setting is obtained from matching ending https://console.cloud.google.com/vertex-ai/locations/us-central1/index-endpoints/3345510418113101824/deployed-indexes/vs_quickstart_deployed_02060053?project=my-project-0004-346516


from google.cloud import aiplatform_v1

# Set variables for the current deployed index.
API_ENDPOINT="63908304.us-central1-255766800726.vdb.vertexai.goog"
INDEX_ENDPOINT="projects/255766800726/locations/us-central1/indexEndpoints/3345510418113101824"
DEPLOYED_INDEX_ID="vs_quickstart_deployed_02060053"
neighbor_count = 10

# Configure Vector Search client
client_options = {
  "api_endpoint": API_ENDPOINT
}
vector_search_client = aiplatform_v1.MatchServiceClient(
  client_options=client_options,
)

# Build FindNeighborsRequest object
datapoint = aiplatform_v1.IndexDatapoint(
  feature_vector=test_embeddings
)
query = aiplatform_v1.FindNeighborsRequest.Query(
  datapoint=datapoint,
  # The number of nearest neighbors to be retrieved
  neighbor_count=neighbor_count
)
request = aiplatform_v1.FindNeighborsRequest(
  index_endpoint=INDEX_ENDPOINT,
  deployed_index_id=DEPLOYED_INDEX_ID,
  # Request can have multiple queries
  queries=[query],
  return_full_datapoint=False,
)

# Execute the request
response = vector_search_client.find_neighbors(request)

# Handle the response
# print(response)

df_new = pd.DataFrame()

for i in range(0,neighbor_count):
    x=response.nearest_neighbors[0]
    # print('id',x.neighbors[i].datapoint.datapoint_id, 'distance',x.neighbors[i].distance)
    
    df_match = df_exploded_2a[df_exploded_2a['id'] == x.neighbors[i].datapoint.datapoint_id]

    # Append the matching rows to the new DataFrame
    df_new = pd.concat([df_new, df_match])

# Print the new DataFrame
print(df_new)


    index                                     pagewise_texts  page_id  \
15      1  WINNIE-THE-POOH\n“Ah, yes, now I do," I said q...        2   
18      1  WINNIE-THE-POOH\n“Ah, yes, now I do," I said q...        2   
0       0  IN WHICH We Are Introduced to\nCHAPTER I\nWinn...        1   
9       1  WINNIE-THE-POOH\n“Ah, yes, now I do," I said q...        2   
8       0  IN WHICH We Are Introduced to\nCHAPTER I\nWinn...        1   
12      1  WINNIE-THE-POOH\n“Ah, yes, now I do," I said q...        2   
21      2  WE ARE INTRODUCED\nAB SANDER\nRNIG\nALSO\n"Win...        3   
1       0  IN WHICH We Are Introduced to\nCHAPTER I\nWinn...        1   
7       0  IN WHICH We Are Introduced to\nCHAPTER I\nWinn...        1   
19      2  WE ARE INTRODUCED\nAB SANDER\nRNIG\nALSO\n"Win...        3   

                                       splitted_texts  \
15  WINNIE-THE-POOH\n“Ah, yes, now I do," I said q...   
18  WINNIE-THE-POOH\n“Ah, yes, now I do," I said q...   
0                        

In [400]:
def get_id_with_embedding_matching(test_embeddings) :
    
    datapoint = aiplatform_v1.IndexDatapoint(
      feature_vector=test_embeddings
    )
    query = aiplatform_v1.FindNeighborsRequest.Query(
      datapoint=datapoint,
      # The number of nearest neighbors to be retrieved
      neighbor_count=neighbor_count
    )
    request = aiplatform_v1.FindNeighborsRequest(
      index_endpoint=INDEX_ENDPOINT,
      deployed_index_id=DEPLOYED_INDEX_ID,
      # Request can have multiple queries
      queries=[query],
      return_full_datapoint=False,
    )

    # Execute the request
    response = vector_search_client.find_neighbors(request)
    
    df_new = pd.DataFrame()

    for i in range(0,neighbor_count):
        x=response.nearest_neighbors[0]
        # print('id',x.neighbors[i].datapoint.datapoint_id, 'distance',x.neighbors[i].distance)

        df_match = df_exploded_2a[df_exploded_2a['id'] == x.neighbors[i].datapoint.datapoint_id]

        # Append the matching rows to the new DataFrame
        df_new = pd.concat([df_new, df_match])

    # Print the new DataFrame
    # print(df_new)
    
    i,j,k = df_new.index[0:3]
    print(i,j,k)
    
    pagewise_texts_v1 = df_new.loc[i, 'pagewise_texts']
    pagewise_texts_v2 = df_new.loc[j, 'pagewise_texts']
    pagewise_texts_v3 = df_new.loc[k, 'pagewise_texts']
    
    splitted_texts_v1 = df_new.loc[i, 'splitted_texts']
    splitted_texts_v2 = df_new.loc[j, 'pagewise_texts']
    splitted_texts_v3 = df_new.loc[k, 'pagewise_texts']
    
    splitted_texts_chunks_v1 = df_new.loc[i, 'splitted_texts_chunks']
    splitted_texts_chunks_v2 = df_new.loc[j, 'splitted_texts_chunks']
    splitted_texts_chunks_v3 = df_new.loc[k, 'splitted_texts_chunks']
    
    page_id_v1 = df_new.loc[i, 'page_id'] 
    page_id_v2 = df_new.loc[j, 'page_id'] 
    page_id_v3 = df_new.loc[k, 'page_id'] 
    
    return(pagewise_texts_v1,pagewise_texts_v2,pagewise_texts_v3,
           splitted_texts_v1,splitted_texts_v2,splitted_texts_v3,
           splitted_texts_chunks_v1,splitted_texts_chunks_v2,splitted_texts_chunks_v3,
        page_id_v1,page_id_v2,page_id_v3,i,j,k)

In [401]:
import pandas as pd
filename = "./harry_potte_qa.csv"
df_qa = pd.read_csv(filename, sep ="|")

df_qa.head()

,Question,Answer
0,What is the name of the magical creature that ...,Thestral
1,What is the name of the school newspaper at Ho...,The Daily Prophet
2,What is the name of the magical map that shows...,Marauder's Map
3,Which Hogwarts house does Luna Lovegood belong...,Ravenclaw
4,What magical creature is known for guarding Gr...,Ukrainian Ironbelly (a dragon)


In [402]:
df_qa.columns

Index(['Question', 'Answer'], dtype='object')

In [410]:
# for i in range(0, len(df_qa)):
#     df_qa.loc[i, "Question_emb"] = embeddings.embed_query( df_qa.loc[i, "Question"])
#     # print("preview embeddings",test_embeddings[0:2])
    
import csv
import csv

with open('harry_potte_qa.csv', 'r') as input_file, open('harry_potte_qa_output.csv', 'w', newline='') as output_file:

  # Create CSV reader and writer objects
  reader = csv.reader(input_file, delimiter='|')
  writer = csv.writer(output_file, delimiter='|')

  # Read and write the header row
  header = next(reader) + ['i','j','k','pagewise_texts_v1','pagewise_texts_v2','pagewise_texts_v3','splitted_texts_v1','splitted_texts_v2','splitted_texts_v3','splitted_texts_chunks_v1','splitted_texts_chunks_v2','splitted_texts_chunks_v3','page_id_v1','page_id_v2','page_id_v3']
  writer.writerow(header)

  # Loop through the remaining rows
  for i, row in enumerate(reader):
    question = row[0].split('|')[0]  # Use 'i' to access the correct element in the row
    question_emb = embeddings.embed_query( question )
    pagewise_texts_v1,pagewise_texts_v2,pagewise_texts_v3,splitted_texts_v1,splitted_texts_v2,splitted_texts_v3,splitted_texts_chunks_v1,splitted_texts_chunks_v2,splitted_texts_chunks_v3,page_id_v1,page_id_v2,page_id_v3,i,j,k = get_id_with_embedding_matching(question_emb) 
    
    # print( i , question)
    row_out = row + [i,j,k,pagewise_texts_v1,pagewise_texts_v2,pagewise_texts_v3,splitted_texts_v1,splitted_texts_v2,splitted_texts_v3,splitted_texts_chunks_v1,splitted_texts_chunks_v2,splitted_texts_chunks_v3,page_id_v1,page_id_v2,page_id_v3]
    
    # Write the row to the output file
    writer.writerow(row_out)

# Usage example:
! head -n 2 harry_potte_qa_output.csv

9 8 19
17 8 9
8 18 0
9 17 8
8 9 19
9 8 15
2 8 19
8 2 13
16 17 1
9 2 8
2 17 0
2 19 15
8 19 2
1 13 8
0 21 16
2 19 8
22 13 2
9 7 19
9 8 10
2 1 9
13 12 15
17 9 8
2 19 13
8 19 2
8 17 13
12 9 13
2 8 17
8 13 19
8 2 19
8 19 13
8 5 19
8 16 17
8 2 9
19 9 8
13 8 2
1 0 21
9 8 7
2 9 1
13 8 9
0 8 9
17 7 8
17 16 8
13 12 8
19 2 8
9 2 7
2 17 23
8 17 2
13 5 12
9 1 7
17 2 8
9 10 17
9 19 1
19 9 8
15 9 17
13 0 8
9 1 19
13 2 8
19 12 8
1 19 12
9 2 8
17 2 8
4 13 8
19 10 12
12 13 1
2 17 12
2 1 13
8 13 19
13 9 12
13 19 8
8 2 17
19 2 12
8 13 9
17 19 8
13 10 14
10 13 9
13 8 12
19 1 10
15 17 19
19 1 12
19 8 1
17 2 5
7 19 12
16 21 0
3 2 19
1 2 17
10 19 4
17 8 2
22 15 9
17 9 2
2 19 17
9 17 2
15 9 1
9 8 19
19 9 6
15 9 19
9 15 19
2 8 19
13 9 15
19 13 17
8 0 2
8 19 9
16 8 17
9 2 17
13 10 9
13 9 8
19 1 2
9 15 13
15 19 7
15 8 19
8 19 16
2 15 9
2 1 17
17 19 2
13 12 8
2 17 1
7 19 2
5 2 8
1 8 16
17 19 15
5 19 17
19 8 17
9 0 22
5 2 17
8 2 12
12 8 13
13 8 14
7 19 2
9 12 13
19 8 2
11 13 5
0 18 10
13 9 12
6 8 19
0 16 19
19 1 16

In [411]:
import pandas as pd
filename = "./harry_potte_qa_output.csv"
df_qa = pd.read_csv(filename, sep ="|")

df_qa.head()

,Question,Answer,i,j,k,pagewise_texts_v1,pagewise_texts_v2,pagewise_texts_v3,splitted_texts_v1,splitted_texts_v2,splitted_texts_v3,splitted_texts_chunks_v1,splitted_texts_chunks_v2,splitted_texts_chunks_v3,page_id_v1,page_id_v2,page_id_v3
0,What is the name of the magical creature that ...,Thestral,9,8,19,"WINNIE-THE-POOH\n“Ah, yes, now I do,"" I said q...",IN WHICH We Are Introduced to\nCHAPTER I\nWinn...,"WE ARE INTRODUCED\nAB SANDER\nRNIG\nALSO\n""Win...","WINNIE-THE-POOH\n“Ah, yes, now I do,"" I said q...",IN WHICH We Are Introduced to\nCHAPTER I\nWinn...,"WE ARE INTRODUCED\nAB SANDER\nRNIG\nALSO\n""Win...","WINNIE-THE-POOH\n“Ah, yes, now I do,"" I said q...","\n""He's Winnie-ther-Pooh. Don't you know what\...","WE ARE INTRODUCED\nAB SANDER\nRNIG\nALSO\n""Win...",2,1,3
1,What is the name of the school newspaper at Ho...,The Daily Prophet,17,8,9,"WINNIE-THE-POOH\n“Ah, yes, now I do,"" I said q...",IN WHICH We Are Introduced to\nCHAPTER I\nWinn...,"WINNIE-THE-POOH\n“Ah, yes, now I do,"" I said q...","WINNIE-THE-POOH\n“Ah, yes, now I do,"" I said q...",IN WHICH We Are Introduced to\nCHAPTER I\nWinn...,"WINNIE-THE-POOH\n“Ah, yes, now I do,"" I said q...","ed\nChristopher Robin.\n""It means he had the n...","\n""He's Winnie-ther-Pooh. Don't you know what\...","WINNIE-THE-POOH\n“Ah, yes, now I do,"" I said q...",2,1,2
2,What is the name of the magical map that shows...,Marauder's Map,8,18,0,IN WHICH We Are Introduced to\nCHAPTER I\nWinn...,"WINNIE-THE-POOH\n“Ah, yes, now I do,"" I said q...",IN WHICH We Are Introduced to\nCHAPTER I\nWinn...,We Are Introduced to\nCHAPTER I\nWinnie-the-Po...,"WINNIE-THE-POOH\n“Ah, yes, now I do,"" I said q...",IN WHICH We Are Introduced to\nCHAPTER I\nWinn...,"\n""He's Winnie-ther-Pooh. Don't you know what\...",Digitized by\nGoogle\n,IN WHICH,1,2,1
3,Which Hogwarts house does Luna Lovegood belong...,Ravenclaw,9,17,8,"WINNIE-THE-POOH\n“Ah, yes, now I do,"" I said q...","WINNIE-THE-POOH\n“Ah, yes, now I do,"" I said q...",IN WHICH We Are Introduced to\nCHAPTER I\nWinn...,"WINNIE-THE-POOH\n“Ah, yes, now I do,"" I said q...","WINNIE-THE-POOH\n“Ah, yes, now I do,"" I said q...",IN WHICH We Are Introduced to\nCHAPTER I\nWinn...,"WINNIE-THE-POOH\n“Ah, yes, now I do,"" I said q...","ed\nChristopher Robin.\n""It means he had the n...","\n""He's Winnie-ther-Pooh. Don't you know what\...",2,2,1
4,What magical creature is known for guarding Gr...,Ukrainian Ironbelly (a dragon),8,9,19,IN WHICH We Are Introduced to\nCHAPTER I\nWinn...,"WINNIE-THE-POOH\n“Ah, yes, now I do,"" I said q...","WE ARE INTRODUCED\nAB SANDER\nRNIG\nALSO\n""Win...",We Are Introduced to\nCHAPTER I\nWinnie-the-Po...,"WINNIE-THE-POOH\n“Ah, yes, now I do,"" I said q...","WE ARE INTRODUCED\nAB SANDER\nRNIG\nALSO\n""Win...","\n""He's Winnie-ther-Pooh. Don't you know what\...","WINNIE-THE-POOH\n“Ah, yes, now I do,"" I said q...","WE ARE INTRODUCED\nAB SANDER\nRNIG\nALSO\n""Win...",1,2,3


In [412]:
# question_emb = embeddings.embed_query( question )

In [413]:
# i,j,k = df_new.index[0:3]
# print(i,j,k)

In [414]:
# get_id_with_embedding_matching(question_emb)